# Introduction

Evaluating retrieval results requires a ground truth or "gold standard" dataset.  
This dataset consists of many queries, and for each query, we know which documents in our knowledge base are relevant.  
In practice, a query may have multiple relevant documents, but for simplicity, we will generate a dataset where each query is linked to one relevant FAQ record.

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Generating Unique IDs for Each Record

To evaluate retrieval, we need a way to uniquely identify each FAQ record.  
We generate an ID for each document by hashing a combination of the course, question, and the first 10 characters of the answer text.  
This approach helps avoid ID collisions even if questions are similar.

In [21]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [ ]:
# Assign IDs to each document using the function above.
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [ ]:
# Inspect one of the processed documents to verify the structure and the assigned ID.
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

## Collision Checking

We import the necessary library to help us check for ID collisions (i.e., different documents with the same ID).

In [ ]:
# Group documents by their generated IDs to check for any collisions.
from collections import defaultdict

hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [ ]:
# Compare number of unique IDs to number of documents. If they are not equal, there are collisions.
len(hashes), len(documents)

(947, 948)

In [ ]:
# Display any IDs that are shared by more than one document, indicating a collision.
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [ ]:
# Inspect a specific collision to understand why it happened.
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [ ]:
# Save the processed documents (with IDs) to a JSON file for later use.
import json

with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [ ]:
# Preview the saved JSON file to ensure it was written correctly.
%head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


## Prompt Template for LLM Question Generation

We define a prompt template for the LLM to generate five user-like questions for each FAQ record.  
The prompt encourages the LLM to create questions that are complete, not too short, and use as few words as possible from the original record.

In [35]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

## Initialize the OpenAI client for generating questions using the prompt template.

In [ ]:
# Initialize the OpenAI client for generating questions using the prompt template.
from openai import OpenAI
client = OpenAI()

In [ ]:
# This function sends the prompt to the LLM and returns the generated questions.
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [ ]:
# Create an empty dictionary to store the generated questions for each document.
from tqdm.auto import tqdm # Import a progress bar

results = {}

## Generate Questions for All Documents

We loop through all documents, generate questions for each using the LLM, and store the results in a dictionary keyed by document ID.  
If the process is interrupted, we can resume without regenerating questions for documents that already have results.

In [ ]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

In [ ]:
# Load previously saved results to avoid regenerating questions and incurring extra costs.
import pickle

with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [ ]:
# Inspect the generated questions for a specific document by its ID.
results['1f6520ca']

'["Where can I find the prerequisites for this course?", "How do I check the prerequisites for this course?", "Where are the course prerequisites listed?", "What are the requirements for joining this course?", "Where is the list of prerequisites for the course?"]'

## Parse LLM Results and Create Document Index

We parse the JSON strings returned by the LLM into Python lists and create a lookup dictionary to quickly access document metadata by ID.

In [ ]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

doc_index = {d['id']: d for d in documents}

## Prepare Final Results for CSV

We prepare a list of tuples containing each generated question, the course, and the relevant document ID.

In [69]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

## Create DataFrame from Final Results and Save to CSV

We create a DataFrame from the final results, with columns for question, course, and document ID. We save the final ground truth dataset as a CSV file, which can be used for evaluating retrieval systems.

In [ ]:
# Import pandas to save as a CSV file
import pandas as pd

df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

df.to_csv('ground-truth-data.csv', index=False)

# Preview the generated CSV file to ensure the data is correctly formatted.
%head ground-truth-data.csv

question,course,document
When does the course begin?,data-engineering-zoomcamp,c02e79ef
How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
Where can I find the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
How do I check the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
Where are the course prerequisites listed?,data-engineering-zoomcamp,1f6520ca
What are the requirements for joining this course?,data-engineering-zoomcamp,1f6520ca
